<a href="https://colab.research.google.com/github/learn-programmers/programmers_kdt_II/blob/main/9%EC%A3%BC%EC%B0%A8_PySpark_%EA%B8%B0%EB%B3%B8_4%EC%9D%BC%EC%B0%A8_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.0.1 py4j==0.10.9 

     |████████████████████████████████| 204.2MB 84kB/s 
     |████████████████████████████████| 204kB 65.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=5a969b07b1e5d19d91947ac8319c79825d894b5ea43a24251ff11d916c9c55ca
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Boston Housing Linear Regression example") \
    .getOrCreate()

# 보스턴 주택 가격 예측 모델 만들기




In [3]:
spark

In [4]:
# training set 다운받기
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv

--2021-02-04 16:57:40--  https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.218.218.105
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.218.218.105|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36240 (35K) [text/csv]
Saving to: ‘boston_housing.csv’

boston_housing.csv  100%[===================>]  35.39K   222KB/s    in 0.2s    

2021-02-04 16:57:41 (222 KB/s) - ‘boston_housing.csv’ saved [36240/36240]



In [5]:
!ls -tl

total 40
drwxr-xr-x 1 root root  4096 Feb  1 17:27 sample_data
-rw-r--r-- 1 root root 36240 Jan 31 01:46 boston_housing.csv


In [8]:
# csv파일을 데이터프레임으로 만들기
data = spark.read.csv('./boston_housing.csv', header=True, inferSchema=True)

In [9]:
data.printSchema() # 스키마(구조) 출력하기
# 마지막 medv가 우리가 예측할 주택의 중간값

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [10]:
data.show() # 앞의 20개 출력

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

## 피쳐 벡터를 만들기

In [13]:
from pyspark.ml.feature import VectorAssembler
# 피쳐를 합쳐주기 위한 VectorAssembler를 임포트한다.

# feature columns는 data의 컬럼에서 마지막 값(medv, 즉 우리가 예측할 값)을 뺀 나머지 저장
feature_columns = data.columns[:-1] 

# VectorAssembler를 사용해 input이 되는 컬럼들(feature_columns)을 묶어서 새로운 컬럼으로 만들고 "features"로 이름을 정한다.
# 칼럼들을 묶어서 하나의 벡터로 만들고 벡터의 칼럼 이름을 feature로 해주는 assembler라는 오브젝트를 만든 것이다.
# 실제로 입력데이터 프레임에 적용하기 전에 그 일을 해줄 오브젝트를 만든 것이다.
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [12]:
feature_columns # 예측할 값 medv를 뺀 나머지 13개의 피쳐들이 잘 들어가 있다.

['crim',
 'zn',
 'indus',
 'chas',
 'nox',
 'rm',
 'age',
 'dis',
 'rad',
 'tax',
 'ptratio',
 'b',
 'lstat']

In [14]:
# 실제 적용은 위에서 만든 어셈블러를 사용한다.
# transform함수를 적용해 입력으로 데이터프레임 data를 주면 출력으로 또다른 데이터프레임이 만들어진다.
# 위에서 만든 assembler의 특성상 새로운 컬럼이 하나 더 생기는 것이다.
# features라는 컬럼에는 벡터 어셈블러의 동작 때문에 지정된 인풋 컬럼들이 하나의 벡터로 묶여서 features라는 하나의 컬럼으로 들어간다.
# transform은 하나의 데이터프레임을 입력으로 받아서 새로운 컬럼을 추가해주는 기능을 한다.
# VectorAssembler는 트랜스포머의 하나로, 지정된 컬럼들을 모아서 새로운 벡터 컬럼을 만드는 일을 한다. 
# 트랜스포머를 만들고 -> transform을 적용하고 인풋으로 데이터프레임을 주는게 기본 형식이다.
data_2 = assembler.transform(data)

In [15]:
data_2.show()
# features라는 새로운 컬럼이 끝에 추가된 것을 볼 수 있다.
# 이 컬럼은 벡터의 형식임을 알 수 있다. 
# 앞의 13개의 모든 컬럼의 값들이 벡터 안에 들어간 것을 볼 수 있다.

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|            features|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|[0.02985,0.0,2.18...|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5

## 훈련용과 테스트용 데이터를 나누고 Linear Regression 모델을 하나 만든다

In [16]:
# 70%는 훈련셋, 30%는 테스트셋
train, test = data_2.randomSplit([0.7, 0.3])

In [18]:
from pyspark.ml.regression import LinearRegression

# 피쳐칼럼을 여러개를 줄 필요 없이 위에서 13가지를 하나로 합친 features 전달 
# 예측해야 할 레이블 값 LabelCol은 medv 전달
# 즉 features, medv의 두개의 필드만 사용
algo = LinearRegression(featuresCol="features", labelCol="medv")

# fit으로 훈련. 위에서 나눈 훈련셋을 인자로 전달
model = algo.fit(train)
# 모델 완성!

## 모델 성능 측정

In [19]:
evaluation_summary = model.evaluate(test)
# 모델 평가는 위에서 만든 테스트셋 데이터프레임 전달해서!
# 결과가 evaluation_summary라는 새로운 데이터프레임에 저장

In [20]:
evaluation_summary # 데이터프레임

In [21]:
# 모델의 성능을 평가하기 위해서는 실제값-예측값을 알아야 한다.
# 다음의 평가방법 결과들이 작아져야 한다.
# 평가방법1
evaluation_summary.meanAbsoluteError

3.200548882491588

In [22]:
# 평가방법2
evaluation_summary.rootMeanSquaredError

4.367919893214391

In [23]:
# 평가방법3
evaluation_summary.r2

0.769243296713928

## 모델 예측값 살펴보기 

In [31]:
# 모델에 transform 적용하여 model에 test에 대하여 예측해 predictions라는 새로운 데이터프레임에 저장
predictions = model.transform(test)

In [30]:
predictions.show()
# 마지막 에 prediction이라는 새로운 칼럼이 생김.

+-------+-----+-----+----+------+-----+----+-------+---+---+-------+------+-----+----+--------------------+------------------+
|   crim|   zn|indus|chas|   nox|   rm| age|    dis|rad|tax|ptratio|     b|lstat|medv|            features|        prediction|
+-------+-----+-----+----+------+-----+----+-------+---+---+-------+------+-----+----+--------------------+------------------+
|0.00632| 18.0| 2.31|   0| 0.538|6.575|65.2|   4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|29.876145971194745|
|0.01096| 55.0| 2.25|   0| 0.389|6.453|31.9| 7.3073|  1|300|   15.3|394.72| 8.23|22.0|[0.01096,55.0,2.2...|26.874994981061604|
|0.01301| 35.0| 1.52|   0| 0.442|7.241|49.3| 7.0379|  1|284|   15.5|394.74| 5.49|32.7|[0.01301,35.0,1.5...|29.487136058357375|
| 0.0136| 75.0|  4.0|   0|  0.41|5.888|47.6| 7.3197|  3|469|   21.1| 396.9| 14.8|18.9|[0.0136,75.0,4.0,...|15.481950096517203|
|0.01381| 80.0| 0.46|   0| 0.422|7.875|32.0| 5.6484|  4|255|   14.4|394.23| 2.97|50.0|[0.01381,80.0,0.4...| 39.

In [32]:
# 표에서 앞의 13개의 피쳐 칼럼들을 세우고
# medv(예측할 값), features(13가지 특징 하나의 벡터로 합친것), prediction(medv 예측값)만 보이자.
# select함수를 사용한다.
predictions.select(predictions.columns[13:]).show()

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|24.0|[0.00632,18.0,2.3...|29.876145971194745|
|22.0|[0.01096,55.0,2.2...|26.874994981061604|
|32.7|[0.01301,35.0,1.5...|29.487136058357375|
|18.9|[0.0136,75.0,4.0,...|15.481950096517203|
|50.0|[0.01381,80.0,0.4...| 39.96820478505988|
|31.6|[0.01432,100.0,1....|32.902087640882456|
|50.0|[0.01501,90.0,1.2...| 43.48607575182313|
|30.1|[0.01709,90.0,2.0...|24.942467233880706|
|32.9|[0.01778,95.0,1.4...| 29.92176803476908|
|50.0|[0.02009,95.0,2.6...| 42.39182156849719|
|24.7|[0.02055,85.0,0.7...|24.661359964448067|
|23.9|[0.02543,55.0,3.7...|27.612973899267484|
|34.7|[0.02729,0.0,7.07...|30.257973659354963|
|17.5|[0.03113,0.0,4.39...| 16.29583669103954|
|34.9|[0.0315,95.0,1.47...| 29.49833882633768|
|34.9|[0.03359,75.0,2.9...| 34.12084495424544|
|22.9|[0.03551,25.0,4.8...|25.408333482097646|
|23.5|[0.03584,80.0,3.3...| 30.36338954827464|
|23.2|[0.0387

In [34]:
# 만든 모델을 저장한다.
# 이렇게 저장하면 나중에 로드해서 다시 쓸 수 있다.
# 문제는 이게 구글 코랩에서 돌리면 내 서버로 다시 다운받기가 불가능하다.
model.save("boston_housing_model")

In [35]:
!ls boston_housing_model

data  metadata


In [36]:
!ls -tl boston_housing_model

total 8
drwxr-xr-x 2 root root 4096 Feb  4 18:18 data
drwxr-xr-x 2 root root 4096 Feb  4 18:18 metadata


In [37]:
# 모델을 구글 드라이브에 저장할 수 있다.
from google.colab import drive
drive.mount('/content/gdrive') # 내 구글 드라이브를 마운트한다.
# url에서 인증코드를 받아서 붙여넣는다.

Mounted at /content/gdrive


In [38]:
# 이제 내 구글 드라이브를 코랩에서 인식할 수 있는 드라이브로 만들었다.
model_save_name = "boston_housing_model"
path = F"/content/gdrive/My Drive/boston_housing_model2" 
model.save(path) # 저장

In [39]:
from pyspark.ml.regression import LinearRegressionModel # 저장했던 모델을 import한다
# 저장했던 모델을 import할때는 LinearRegressionModel이라는 모듈을 임포트해야 한다.

#저장했던 path를 입력으로 준다
loaded_model = LinearRegressionModel.load(path)  # "boston_housing_model")

In [42]:
predictions2 = loaded_model.transform(test)
# 로딩된 모델이 제대로 동작하는지 test데이터셋을 넣어 예측값을 출력해본다.

In [41]:
predictions2.select(predictions.columns[13:]).show()
# 모델이 제대로 저장됐고 정상적으로 로딩되어 쓰임을 알 수 있다.

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|24.0|[0.00632,18.0,2.3...|29.876145971194745|
|22.0|[0.01096,55.0,2.2...|26.874994981061604|
|32.7|[0.01301,35.0,1.5...|29.487136058357375|
|18.9|[0.0136,75.0,4.0,...|15.481950096517203|
|50.0|[0.01381,80.0,0.4...| 39.96820478505988|
|31.6|[0.01432,100.0,1....|32.902087640882456|
|50.0|[0.01501,90.0,1.2...| 43.48607575182313|
|30.1|[0.01709,90.0,2.0...|24.942467233880706|
|32.9|[0.01778,95.0,1.4...| 29.92176803476908|
|50.0|[0.02009,95.0,2.6...| 42.39182156849719|
|24.7|[0.02055,85.0,0.7...|24.661359964448067|
|23.9|[0.02543,55.0,3.7...|27.612973899267484|
|34.7|[0.02729,0.0,7.07...|30.257973659354963|
|17.5|[0.03113,0.0,4.39...| 16.29583669103954|
|34.9|[0.0315,95.0,1.47...| 29.49833882633768|
|34.9|[0.03359,75.0,2.9...| 34.12084495424544|
|22.9|[0.03551,25.0,4.8...|25.408333482097646|
|23.5|[0.03584,80.0,3.3...| 30.36338954827464|
|23.2|[0.0387